---
Author: Mustapha Bouhsen <br>
[LinkedIn](https://www.linkedin.com/in/mustapha-bouhsen/)<br>
[Git](https://github.com/mus514)<br>
Date: February 14, 2024<br>
---

In [0]:
%run Repos/bouhsen.m@gmail.com/ML_Pipeline_Hub/library/garch_model

In [0]:
%run Repos/bouhsen.m@gmail.com/ML_Pipeline_Hub/library/daily_utilities

In [0]:
#-----------------------------------------
# Set the table folder path
#-----------------------------------------
tables_folder_path = "/mnt/tables/"

## Join stocks tables

In [0]:
%sql
SELECT
    date, open, high, low, close, adj_close, volume
FROM
aapl
LIMIT 10
    

date,open,high,low,close,adj_close,volume
2004-01-02,0.384821,0.388393,0.378214,0.38,0.32170784,1.446424E8
2004-01-05,0.3825,0.399821,0.3825,0.395893,0.33516276,3.950184E8
2004-01-06,0.397321,0.400357,0.387679,0.394464,0.33395284,5.09348E8
2004-01-07,0.394643,0.407679,0.391607,0.403393,0.3415123,5.8687437E8
2004-01-08,0.407857,0.42375,0.404464,0.417143,0.35315302,4.603032E8
2004-01-09,0.414821,0.430893,0.406964,0.410714,0.34771028,4.274592E8
2004-01-12,0.415179,0.428571,0.4125,0.42375,0.35874656,4.875472E8
2004-01-13,0.441071,0.443571,0.426071,0.430714,0.36464235,6.7901683E8
2004-01-14,0.435714,0.438214,0.424643,0.432143,0.3658521,6.200432E8
2004-01-15,0.409107,0.417857,0.401786,0.408036,0.34544307,1.01820877E9


In [0]:
%sql
/***************************************
*
* Creating join table for stocks information
* 
****************************************/
DROP TABLE IF EXISTS stocks;

CREATE TABLE stocks AS

SELECT
    date, open, high, low, close, adj_close, volume,
    'aapl' AS stock
FROM
    aapl

UNION

SELECT
    date, open, high, low, close, adj_close, volume,
    'amzn' AS stock
FROM
    amzn

UNION

SELECT
    date, open, high, low, close, adj_close, volume,
    'googl' AS stock
FROM
    googl

UNION

SELECT
    date, open, high, low, close, adj_close, volume,
    'msft' AS stock
FROM
    msft

num_affected_rows,num_inserted_rows


In [0]:
#-----------------------------------------
# Loading the stocks table and save it in csv
#-----------------------------------------
df = spark.sql("SELECT * FROM stocks")


# Temp folder to save temp parquet files
temp_folder = tables_folder_path + f"temp/"

# write data frame to csv
df.coalesce(1).write.mode("overwrite").option("header", "True").csv(temp_folder)

#get all files path ending with .parquet
files_paths = get_files_paths_from_folders(temp_folder, ".csv")
            
# Copy parquet files to final destination
ingest_and_transform_to_parquet(files_paths, tables_folder_path, "stocks")

# delete the temp folder
delete_contents_recursively(temp_folder)

## Join Returns tables

In [0]:
%sql
/***************************************
*
* Creating join table for returns
* 
****************************************/
DROP TABLE IF EXISTS returns;

CREATE TABLE returns AS

SELECT 
    date,
    return,
    stock
FROM (
    SELECT 
        date,
        aapl,
        amzn,
        googl,
        msft
    FROM stocks_returns
) AS Source
UNPIVOT (
    Return FOR stock IN (aapl, amzn, googl, msft)
) AS UnpivotedTable;

num_affected_rows,num_inserted_rows


In [0]:
#-----------------------------------------
# Loading the stocks table and save it in csv
#-----------------------------------------
df = spark.sql("SELECT * FROM returns")


# Temp folder to save temp parquet files
temp_folder = tables_folder_path + f"temp/"

# write data frame to csv
df.coalesce(1).write.mode("overwrite").option("header", "True").csv(temp_folder)

#get all files path ending with .parquet
files_paths = get_files_paths_from_folders(temp_folder, ".csv")
            
# Copy parquet files to final destination
ingest_and_transform_to_parquet(files_paths, tables_folder_path, "returns")

# delete the temp folder
delete_contents_recursively(temp_folder)

## Join Volatilities tables

In [0]:
%sql
/***************************************
*
* Creating join table for volatilities
* 
****************************************/
DROP TABLE IF EXISTS volatilities;

CREATE TABLE volatilities AS

SELECT 
    date,
    volatility,
    stock
FROM (
    SELECT 
        date,
        aapl,
        amzn,
        googl,
        msft
    FROM stocks_volatility
) AS Source
UNPIVOT (
    volatility FOR stock IN (aapl, amzn, googl, msft)
) AS UnpivotedTable;

num_affected_rows,num_inserted_rows


In [0]:
#-----------------------------------------
# Loading the volatility table and save it in csv
#-----------------------------------------
df = spark.sql("SELECT * FROM volatilities")


# Temp folder to save temp parquet files
temp_folder = tables_folder_path + f"temp/"

# write data frame to csv
df.coalesce(1).write.mode("overwrite").option("header", "True").csv(temp_folder)

#get all files path ending with .parquet
files_paths = get_files_paths_from_folders(temp_folder, ".csv")
            
# Copy parquet files to final destination
ingest_and_transform_to_parquet(files_paths, tables_folder_path, "volatilities")

# delete the temp folder
delete_contents_recursively(temp_folder)

## Join Monte-Carlo simulation tables

In [0]:
tables = ["aapl_simulation", "amzn_simulation", "googl_simulation", "msft_simulation"]

In [0]:
prod_folder_path = "/dbfs/mnt/prod/"
for stock in tables:
    # improt data
    df = pd.read_csv(prod_folder_path + stock + ".csv")

    df_avr = pd.DataFrame()
    # Get the last price of the stock
    df_avr["date"] =df["date"]

    # average 
    df_avr["adj_stock"] = df.drop('date', axis=1).values.mean(axis=1)

    # Convert simulated prices to a Spark DataFrame with the specified schema
    df = spark.createDataFrame(df_avr)

    # Define the table name for the Monte Carlo simulation results
    table_name = f'{stock}_avr'

    # Check if the table already exists in the Spark catalog
    if spark.catalog.tableExists(f"{table_name}"):
        # If the table exists, drop it
        spark.sql(f"DROP TABLE {table_name}")
        print(f'Dropped table: {table_name}')

    # Create a new table with the simulated prices
    df.write.format("parquet").saveAsTable(table_name) 

Dropped table: aapl_simulation_avr
Dropped table: amzn_simulation_avr
Dropped table: googl_simulation_avr
Dropped table: msft_simulation_avr


In [0]:
%sql
/***************************************
*
* Creating join table for Monte-Carlo information
* 
****************************************/

DROP TABLE IF EXISTS simulation_avr;

CREATE TABLE simulation_avr AS

SELECT
    *,
    'aapl' AS stock
FROM
    aapl_simulation_avr

UNION

SELECT
    *,
    'amzn' AS stock
FROM
    amzn_simulation_avr

UNION

SELECT
    *,
    'googl' AS stock
FROM
    googl_simulation_avr

UNION

SELECT
    *,
    'msft' AS stock
FROM
    msft_simulation_avr

num_affected_rows,num_inserted_rows


In [0]:
#-----------------------------------------
# Loading the simulation monte-carlo table and save it in csv
#-----------------------------------------
df = spark.sql("SELECT * FROM simulation_avr")


# Temp folder to save temp parquet files
temp_folder = tables_folder_path + f"temp/"

# write data frame to csv
df.coalesce(1).write.mode("overwrite").option("header", "True").csv(temp_folder)

#get all files path ending with .parquet
files_paths = get_files_paths_from_folders(temp_folder, ".csv")
            
# Copy parquet files to final destination
ingest_and_transform_to_parquet(files_paths, tables_folder_path, "simulation_avr")

# delete the temp folder
delete_contents_recursively(temp_folder)

## Join Monte-Carlo simulation of the last prices tables

In [0]:
%sql
/***************************************
*
* Creating join table for volatilities
* 
****************************************/
DROP TABLE IF EXISTS last_prices;

CREATE TABLE last_prices AS

SELECT 
    simulation,
    stock
FROM (
    SELECT 
        aapl,
        amzn,
        googl,
        msft
    FROM simulation_of_last_prices
) AS Source
UNPIVOT (
    simulation FOR stock IN (aapl, amzn, googl, msft)
) AS UnpivotedTable;

num_affected_rows,num_inserted_rows


In [0]:
#-----------------------------------------
# Loading the last prices simulations table and save it in csv
#-----------------------------------------
df = spark.sql("SELECT * FROM last_prices")


# Temp folder to save temp parquet files
temp_folder = tables_folder_path + f"temp/"

# write data frame to csv
df.coalesce(1).write.mode("overwrite").option("header", "True").csv(temp_folder)

#get all files path ending with .parquet
files_paths = get_files_paths_from_folders(temp_folder, ".csv")
            
# Copy parquet files to final destination
ingest_and_transform_to_parquet(files_paths, tables_folder_path, "last_prices")

# delete the temp folder
delete_contents_recursively(temp_folder)